In [ ]:
using System.IO;
using System.Text.Json.Serialization;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Threading;

public static class Settings
{
    private static readonly ReaderWriterLockSlim locker = new();

    public static JsonElement ROUND_INFO { get; set; }
    public static JsonElement ROUND_STRADDLE_INFO { get; set; }
    public static JsonNode SETTINGS { get; set; }

    public static void LoadSettings(string settings_path, string round_path, string round_straddle_path)
    {
        ROUND_INFO = JsonDocument.Parse(File.ReadAllText(Environment.CurrentDirectory + "\\" + round_path)).RootElement;
        ROUND_STRADDLE_INFO = JsonDocument.Parse(File.ReadAllText(Environment.CurrentDirectory + "\\" + round_straddle_path)).RootElement;
        SETTINGS = JsonNode.Parse(File.ReadAllText(Environment.CurrentDirectory + "\\" + settings_path));
    }

    public static void WriteSettings(string pathToSettingsJsonFile)
    {
        try
        {
            locker.EnterWriteLock();
            JsonSerializerOptions options = new JsonSerializerOptions { WriteIndented = true };
            File.WriteAllText(Path.Combine(Environment.CurrentDirectory, pathToSettingsJsonFile), JsonSerializer.Serialize(SETTINGS, options));
        }
        finally
        {
            locker.ExitWriteLock();
        }
    }
}

// Settings.SETTINGS = JsonDocument.Parse("{\"name\":\"John Smith\",\"age\":35}").RootElement;
// Settings.WriteSettings("test.json");
Settings.LoadSettings("test.json", "round_straddle.json", "round_straddle.json");
// Console.WriteLine(Settings.SETTINGS.GetProperty("name").ToString());



In [ ]:
 public class GameHeroCards
    {
        [JsonPropertyName("Card1")]
        public string Card1;

        [JsonPropertyName("Card2")]
        public string Card2;
    }

    public class GameSeat
    {
        [JsonPropertyName("Seat")]
        public int Seat { get; set; }

        [JsonPropertyName("UUID")]
        public int UUID { get; set; }

        [JsonPropertyName("sID")]
        public string sID { get; set; }

        [JsonPropertyName("GState")]
        public int GState { get; set; }

        [JsonPropertyName("GameRole")]
        public int GameRole { get; set; }

        [JsonPropertyName("Stack")]
        public double Stack { get; set; }

        [JsonPropertyName("PAnte")]
        public int PAnte { get; set; }

        [JsonPropertyName("isA")]
        public int isA { get; set; }

        [JsonPropertyName("Action")]
        public string Action { get; set; }
    }

    public class LogcatJsonEntry
    {
        [JsonPropertyName("handID")]
        public int handID { get; set; }

        [JsonPropertyName("roomGameState")]
        public int roomGameState { get; set; }

        [JsonPropertyName("heroID")]
        public int heroID { get; set; }

        [JsonPropertyName("bigBlind")]
        public int bigBlind { get; set; }

        [JsonPropertyName("seats")]
        public List<GameSeat> seats { get; set; }

        [JsonPropertyName("heroCards")]
        public GameHeroCards heroCards { get; set; }

        [JsonPropertyName("heroGameState")]
        public string heroGameState { get; set; }
    }

In [ ]:
public static double Round_Stack(int players_count, double stack, bool withStraddle)
{
    
    var array = withStraddle ? Settings.ROUND_STRADDLE_INFO : Settings.ROUND_INFO;
    var element = array.GetProperty(players_count.ToString()).EnumerateArray()
        .FirstOrDefault(value => stack >= value.GetProperty("min").GetDouble() && stack <= value.GetProperty("max").GetDouble());
    return element.ValueKind != JsonValueKind.Null ? element.GetProperty("value").GetDouble() : 777;
}

// Console.WriteLine(Round_Stack(6, 15, true));
// Console.WriteLine(Round_Stack(3, 211, true));

In [ ]:
double bb_multi = 1.0;

string jsonString = "{\"heroID\":3641664,\"heroGameState\":\"\",\"heroCards\":{},\"roomGameState\":7,\"seats\":[{\"sID\":\"6007696769\",\"UUID\":2462703,\"GameRole\":1,\"isA\":1,\"PAnte\":400,\"Stack\":0,\"Action\":\"R\",\"GState\":10,\"Seat\":3},{\"sID\":\"7121840883\",\"UUID\":2917423,\"GameRole\":2,\"isA\":0,\"PAnte\":420,\"Stack\":8440,\"Action\":\"R\",\"GState\":10,\"Seat\":4},{\"sID\":\"8869944905\",\"UUID\":3635269,\"GameRole\":3,\"isA\":0,\"PAnte\":440,\"Stack\":7560,\"Action\":\"F\",\"GState\":5,\"Seat\":5}],\"bigBlind\":40,\"handID\":41}";

LogcatJsonEntry? jsonEntry = JsonSerializer.Deserialize<LogcatJsonEntry>(jsonString);

Console.WriteLine(jsonEntry.seats.Select(seat => { if (seat.GameRole==1) { return seat;} else{return null;}}).ToList<GameSeat>()[0]);

bool withStraddle = jsonEntry.seats.Any(seat => seat.GameRole == 5);
jsonEntry.seats = jsonEntry.seats.Select(seat =>
{
    seat.Stack = seat.Stack != 0 ? Round_Stack(jsonEntry.seats.Count, Math.Round((seat.Stack / jsonEntry.bigBlind) * bb_multi), withStraddle) : 0;
    return seat;
}).ToList();

jsonEntry.seats = jsonEntry.seats.Select(seat =>
{
    seat.PAnte /= jsonEntry.bigBlind;
    return seat;
}).ToList();

List<GameSeat> temp_sort_stack_seats = new(jsonEntry.seats.ToArray());
temp_sort_stack_seats.Sort((p, q) => p.Stack.CompareTo(q.Stack));
temp_sort_stack_seats.Reverse();
jsonEntry.seats.Find(x => x.UUID == temp_sort_stack_seats[0].UUID).Stack = temp_sort_stack_seats[1].Stack;
temp_sort_stack_seats = null;








In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Collections.Generic;
using System.Data;
using System.IO;

var dt = new DataTable();
dt.Columns.Add(new DataColumn("ActNum", typeof(int)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("sID", typeof(long)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("Stack", typeof(int)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("PAnte", typeof(double)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("isA", typeof(int)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("Action", typeof(string)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("Cards", typeof(string)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("Hero", typeof(string)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("HandID", typeof(int)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("Ev", typeof(string)) { AllowDBNull = true });
dt.Columns.Add(new DataColumn("Pb", typeof(int)) { AllowDBNull = true });

dt.PrimaryKey = new DataColumn[] { dt.Columns["sID"] };
dt.DefaultView.Sort = "ActNum ASC";

int meHeroIndex = 2;

List<DataRow> orderedList = new List<DataRow>();

DataRow row1 = dt.NewRow();
row1["ActNum"] = 1;
row1["sID"] = 11;
row1["Stack"] = 10;
row1["PAnte"] = 2.5;
row1["isA"] = 1;
row1["Action"] = "R";
row1["Cards"] = "AsKs";
row1["Hero"] = "";
row1["HandID"] = 1;
row1["Ev"] = "0.5";
row1["Pb"] = 70;
orderedList.Add(row1);

DataRow row2 = dt.NewRow();
row2["ActNum"] = 2;
row2["sID"] = 12;
row2["Stack"] = 15;
row2["PAnte"] = 0.0;
row2["isA"] = 0;
row2["Action"] = "R";
row2["Cards"] = "Qh9h";
row2["Hero"] = "Hero";
row2["HandID"] = 1;
row2["Ev"] = "-0.5";
row2["Pb"] = 30;
orderedList.Add(row2);

//-- current
DataRow row3 = dt.NewRow();
row3["ActNum"] = 3;
row3["sID"] = 13;
row3["Stack"] = 20;
row3["PAnte"] = 0.0;
row3["isA"] = 0;
row3["Action"] = "";
row3["Cards"] = "5c5d";
row3["Hero"] = "Hero";
row3["HandID"] = 1;
row3["Ev"] = "0.0";
row3["Pb"] = 0;
orderedList.Add(row3);

DataRow row4 = dt.NewRow();
row4["ActNum"] = 4;
row4["sID"] = 14;
row4["Stack"] = 20;
row4["PAnte"] = 0.0;
row4["isA"] = 1;
row4["Action"] = "";
row4["Cards"] = "";
row4["Hero"] = "";
row4["HandID"] = 1;
row4["Ev"] = "0.5";
row4["Pb"] = 70;
orderedList.Add(row4);

DataRow row5 = dt.NewRow();
row5["ActNum"] = 5;
row5["sID"] = 15;
row5["Stack"] = 15;
row5["PAnte"] = 0.0;
row5["isA"] = 0;
row5["Action"] = "R";
row5["Cards"] = "Jc8s";
row5["Hero"] = "Hero";
row5["HandID"] = 1;
row5["Ev"] = "-0.5";
row5["Pb"] = 30;
orderedList.Add(row5);

DataRow row6 = dt.NewRow();
row6["ActNum"] = 6;
row6["sID"] = 16;
row6["Stack"] = 10;
row6["PAnte"] = 2.5;
row6["isA"] = 0;
row6["Action"] = "";
row6["Cards"] = "";
row6["Hero"] = "";
row6["HandID"] = 1;
row6["Ev"] = "0.0";
row6["Pb"] = 0;
orderedList.Add(row6);


// Пройдіться по всіх рядках таблиці та виведіть значення кожної колонки
foreach (DataRow row in orderedList)
{
    foreach (DataColumn col in dt.Columns)
    {
        Console.Write(row[col] + "\t");
    }
    Console.WriteLine();
}

// Виведіть роздільник між рядками
Console.WriteLine("---------------------------------------------");



In [ ]:
// GET SUBSET FROM ME HERO TO BOTTOM OF NOT HEROES WITH IT ACT NUMS
List<DataRow> bottomNonActionPlayers = orderedList.Skip(meHeroIndex).Where(dataRow => dataRow.Field<string>("Action") == "").ToList();
List<int> actNumIndexes = bottomNonActionPlayers.Select(row =>  int.Parse(row.ItemArray[0].ToString())).ToList();                                    

// DEBUG
foreach (DataRow row in bottomNonActionPlayers)
{
    foreach (DataColumn col in row.Table.Columns)
    {
        Console.Write(row[col] + "\t");
    }
    Console.WriteLine();
}

Console.WriteLine(string.Join(", ", actNumIndexes));

In [ ]:
// GENERATION ALL POSIBLE ACTION VARIANTS
private static void GetAllPossibleActions(List<List<string>> result, List<string> temp, int length)
{
    if (temp.Count == length)
    {
        result.Add(new List<string>(temp));
        return;
    }

    foreach (string option in new string[] { "R", "F" })
    {
        temp.Add(option);
        GetAllPossibleActions(result, temp, length);
        temp.RemoveAt(temp.Count - 1);
    }
}

List<List<string>> actionVariants = new List<List<string>>();    
GetAllPossibleActions(actionVariants, new List<string>(), bottomNonActionPlayers.Count());

// DEBUG
Console.WriteLine("All possible combinations (Count): " + actionVariants.Count);
foreach (List<string> actionVariants in actionVariants)
{
    Console.WriteLine(String.Join(", ", actionVariants));
}

In [ ]:
using System.Reflection;

public static List<List<DataRow>> FillAllPossibleTables(List<List<string>> actionVariants, List<DataRow> origTable, List<int> replaceRowIndexes)
{
    var result = new List<List<DataRow>>();

    foreach(var actionVariant in actionVariants){ // actionVariant = { r, f , f}
        
        // deep copy orig table
        List<DataRow> variantTable = new List<DataRow>();

        foreach (DataRow row in origTable)
        {
            DataRow clonedRow = row.Table.NewRow(); // створення нового DataRow
            clonedRow.ItemArray = row.ItemArray.Clone() as object[]; // глибока копія значень властивостей
            variantTable.Add(clonedRow);
        }
        //------------------------

        Console.WriteLine("actionVariant: " +  string.Join(", ",actionVariant));

        foreach( var (replaceRowIndex, index) in replaceRowIndexes.Select((value, i) => (value, i))){
            
            variantTable.FirstOrDefault(dataRow => dataRow.Field<int>("ActNum") == replaceRowIndex)["Action"] = actionVariant[index];
            
            // Console.WriteLine("origTable row at index (" + index + ")    " + string.Join(", ", replaceRowIndex)            
            // + "     variantTable Action: " + variantTable.FirstOrDefault(dataRow => dataRow.Field<int>("ActNum") == replaceRowIndex)["Action"]
            
            // );
        }
        result.Add(variantTable);
    }

    return result;
}

List<List<DataRow>> res = FillAllPossibleTables(actionVariants, orderedList, actNumIndexes);


// string HandID = "12";
// string filePath = @""+HandID+"\\Tables.txt";
// string directoryPath = Path.GetDirectoryName(filePath);
// if (!Directory.Exists(directoryPath))
// {
//     Directory.CreateDirectory(directoryPath);
// }

// // Відкриваємо файл для запису
// using (StreamWriter writer = new StreamWriter(filePath))
// {
//     // DEBUG
//     foreach (List<DataRow> list in res)
//     {
//         writer.WriteLine("----------------------------------------------------------------\n");
//         var dt1 = new DataTable();
//         dt1.Columns.Add(new DataColumn("ActNum", typeof(int)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("sID", typeof(long)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("Stack", typeof(int)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("PAnte", typeof(double)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("isA", typeof(int)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("Action", typeof(string)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("Cards", typeof(string)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("Hero", typeof(string)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("HandID", typeof(int)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("Ev", typeof(string)) { AllowDBNull = true });
//         dt1.Columns.Add(new DataColumn("Pb", typeof(int)) { AllowDBNull = true });
        
//         foreach (var (row, index) in list.Select((value, i) => (value, i)))
//         {
//             writer.WriteLine($"{row["ActNum"],-1}, {row["sID"],-2}, {row["Stack"],-2}, {row["PAnte"],-3}, {row["isA"],-5}, {row["Action"],-5}, {row["Cards"],-5}, {row["Hero"],-5}, {row["HandID"],-5}, {row["Ev"],-5}, {row["Pb"],-5}");
//         }
//         writer.WriteLine("----------------------------------------------------------------\n");
//     }
// }


// Console.WriteLine("Count: " + res.Count);
// Console.WriteLine(string.Join(", ", actNumIndexes));

// DEBUG
foreach (List<DataRow> list in res)
{
    Console.WriteLine("----------------------------------------------------------------\n");
    var dt1 = new DataTable();
    dt1.Columns.Add(new DataColumn("ActNum", typeof(int)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("sID", typeof(long)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("Stack", typeof(int)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("PAnte", typeof(double)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("isA", typeof(int)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("Action", typeof(string)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("Cards", typeof(string)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("Hero", typeof(string)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("HandID", typeof(int)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("Ev", typeof(string)) { AllowDBNull = true });
    dt1.Columns.Add(new DataColumn("Pb", typeof(int)) { AllowDBNull = true });
    
    foreach (var (row, index) in list.Select((value, i) => (value, i)))
    {
        if (actNumIndexes.Contains(index+1)){
            Console.WriteLine($"\u001b[31m{row["ActNum"],-1}, {row["sID"],-2}, {row["Stack"],-2}, {row["PAnte"],-3}, {row["isA"],-5}, {row["Action"],-5}, {row["Cards"],-5}, {row["Hero"],-5}, {row["HandID"],-5}, {row["Ev"],-5}, {row["Pb"],-5}\u001b[0m");

        }else{
            Console.WriteLine($"{row["ActNum"],-1}, {row["sID"],-2}, {row["Stack"],-2}, {row["PAnte"],-3}, {row["isA"],-5}, {row["Action"],-5}, {row["Cards"],-5}, {row["Hero"],-5}, {row["HandID"],-5}, {row["Ev"],-5}, {row["Pb"],-5}");

        }
    }

    Console.WriteLine("----------------------------------------------------------------\n");

}


In [ ]:

using System;
using System.Collections.Generic;
using System.Runtime.InteropServices;

 [DllImport(@"C:\DEV\POKER\PokerKingPrev\PokerKingTeam\OMPEvalLib\x64\Debug\OMPEvalLib.dll", CallingConvention = CallingConvention.StdCall)]
        public static extern IntPtr Calculate(string[] cardRanges,
                                            int cardsCount,
                                            string boardCard,
                                            string dCard,
                                            double stdErrMarginPercent,
                                            double calculationTimeSec,
                                            double updateInterval,
                                            int threads,
                                            bool useEnumerateInsteadMonteCarlo
                                          );


double  firstPot = 32; // 0
string dCard = "5h5c5d5s"; // ""

//-- EQUITY actNum - stackStart - stackEnd_plus_num - cards
List<(int actNum, double stackStart, double stackEnd_plus_num, string cards)> stackStartStackEndPairs = new(){
    (1, 70, 0, "As2s"),
    (2, 40, 0, "KdKs"),
    (3, 100, 0, "3s3d")
};

int round = 1; // first round
bool anyMoreZero = stackStartStackEndPairs.Any(p => p.Item2 > 0); // if have stacks > 0
bool run = true;

// запускаем цикл пока чей-то стек больше 0
while (anyMoreZero && run ){
    double pot = 0; // bank

    // если это первый банк добавляем в pot firstPot
    if (round==1){
        pot += firstPot;
    }
    
    // если в цикле остался 1 игрок, переливаем его стекСтарт в стекЕнд + pot
    int countPlayers = 0;
    int num = 0;

    foreach (var stack in stackStartStackEndPairs){

        num = stack.Item1; // actNum
        if (stack.Item2 !=0){ // stackStart
            countPlayers+=1;
        }
       
    }

    //-- if only 1 element in stackStartStackEndPairs
    if (countPlayers==1){
        var pair = stackStartStackEndPairs.FirstOrDefault(p => p.Item2 > 0 );
        // stackEnd_plus_num    stackStart
        pair.Item3            = pair.Item2 + pot;
        run = false;
        break;
    }

    // Определяем наименьший стек with != 0
    var stackMin = stackStartStackEndPairs.Where(p => p.Item2 != 0)
                     .OrderBy(p => p.Item2)
                     .Select(p => p.Item3)
                     .FirstOrDefault();

    // Формируем пот и кидаем карты в калькулятор
    List<(int actNum, double stackStart, double stackEnd_plus_num, string cards)> modifiedPairs = new();
    foreach (var pair in stackStartStackEndPairs)
    {
           // stackStart
        if (pair.Item2 != 0){ 
            pot += stackMin;             // stackStart
            modifiedPairs.Add((pair.Item1, pair.Item2 - stackMin, pair.Item3, pair.Item4));
        }else{
            modifiedPairs.Add((pair.Item1, pair.Item2, pair.Item3, pair.Item4));
        }
    }
    stackStartStackEndPairs = modifiedPairs;


    // filter elements with stackStar != 0
    List<(int actNum, double stackStart, double stackEnd_plus_num, string cards)> elementsForEquity = stackStartStackEndPairs.Where(p => p.Item2 != 0).ToList();
    
    // cards list for equity calc
    List<string> cardsListEquity = elementsForEquity.Select(p => p.Item4).ToList<string>();

    //-- CALL CALC
    // ADD CARDS, ADD dCard, 
    // CALC EQ 

    var res = Calculate(cardRanges: cardsListEquity.ToArray(),          // card ranges list
                                cardsCount: cardsListEquity.Count,      // card ranges count
                                boardCard: "",                          // board card mask
                                dCard: dCard,                           // d card
                                stdErrMarginPercent: 0.02,              // stop calcualte when standard error below 0.002% 
                                calculationTimeSec: 1.0,                // calcualte during 1 sec
                                updateInterval: 0.25,                   // 0.25 sec update interval
                                threads: 0,                             // max
                                useEnumerateInsteadMonteCarlo: false    // use enumerate instead montecarlo
                                );

    double[] returnArray = new double[6];
    Marshal.Copy(res, returnArray, 0, 6);

    // задаем стекЕнд по еквити
    List<(int actNum, double stackStart, double stackEnd_plus_num, string cards)> modifiedPairs2 = new();

    foreach (var (pair, index) in stackStartStackEndPairs.Select((value, i) => (value, i)))
    {
                                                         // stackEnd
        modifiedPairs2.Add((pair.Item1, pair.Item2, pot * returnArray[index], pair.Item4));
    }
    stackStartStackEndPairs = modifiedPairs2;

    round+=1;
}







In [96]:
//--------------------------------------------------------------- COINS
#r "nuget:Deedle"
#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget: Deedle.DotNet.Interactive.Extension,0.1.0-alpha9"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:XPlot.Plotly"


Restore sources https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json Installed Packages Deedle, 3.0.0 Deedle.DotNet.Interactive.Extension, 0.1.0-alpha9 Microsoft.Data.Analysis, 0.4.0 XPlot.Plotly, 4.0.6

Loading extensions from `C:\Users\Lion\.nuget\packages\deedle.dotnet.interactive.extension\0.1.0-alpha9\interactive-extensions\dotnet\Deedle.DotNet.Interactive.Extension.dll`

Added DeedleFormatterExtension including formatters for Frame and Series

In [127]:
using System.IO;
using System.Globalization;
using System.Linq;
using Deedle;
using Deedle.Vectors;
using XPlot.Plotly;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;

// CSV-файл
var csvPath = "btcusdt_datetime.csv";

// Зчитуємо дані з CSV файлу та зберігаємо їх у Data.Frame
var csv = Frame.ReadCsv(
    csvPath,
    separators: ",",
    hasHeaders: true
);

csv.RenameColumns(new string[]
{
    "open","close","high","low","volume","time"
});

csv.IndexRows<DateTime>("time");






// var dateTimes = csv.Rows.Select(row => DateTimeOffset.FromUnixTimeMilliseconds(row.Value.GetAs<Int64>("time")).UtcDateTime );
// // Replace the original timestamp column with the new DateTime column
// csv.ReplaceColumn("time", dateTimes);
// // Export to CSV
// using (var writer = new StreamWriter("btcusdt_datetime.csv"))
// {
//     csv.SaveCsv(writer);
// }


// return csv;
// var priceSeries = csv.GetColumn<double>("Price");
// var priceDiffSeries = priceSeries - priceSeries.Shift(1).FillMissing(0);
// frame.AddColumn("PriceDiff", priceDiffSeries);



// // Додаємо новий стовпець з різницею в ціні між попереднім днем
 
// // Перетворюємо стовпець цін в числа з плаваючою точкою
// var prices = csv.Rows.Select(row => row.Value.GetAs<double>("price"));
// // Створюємо новий стовпець, заповнюємо перший елемент нулем
// var priceDifferences = new List<double> { 0 };
// // Рахуємо різницю між ціною сьогодні та ціною вчора та додаємо в новий стовпець
// for (int i = 1; i < prices.ValueCount; i++)
// {
//     var difference = prices[i] - prices[i-1];
//     priceDifferences.Add(difference);
// }
// csv.AddColumn("PriceChange", priceDifferences);


// var grouped = csv.GroupRowsBy<DateTime>("Date");
// return grouped;

// Create a new frame with the results
// var result = Frame.FromColumns(grouped);

    // .Select(group =>
    // {
    //     var date = group.Key;
    //     var maxPrice = group.Max(row => row.GetAs<double>("Price"));
    //     var minPrice = group.Min(row => row.GetAs<double>("Price"));
    //     return Row.FromTuple(date, maxPrice, minPrice);
    // })
    // .SortRowsByKey();




// Виводимо кількість рядків у DataFrame
Console.WriteLine($"Кількість рядків: {csv.RowCount}");
// Plot
// return csv;


Кількість рядків: 4341245


In [ ]:
 // Групуємо дані за днем та п'ятихвилинним інтервалом

        // Групуємо дані за днем та п'ятихвилинним інтервалом за допомогою LINQ
        var grouped = csv.Rows
            .GroupBy(row => new
            {
                Date = row.Value.GetAs<DateTime>("time").Date,
                Interval = TimeSpan.FromMinutes(5) * (int)(row.Value.GetAs<DateTime>("time").TimeOfDay.TotalMinutes / 5)
            });
            return grouped;
            // .Select(group =>
            // {
            //     var high = group.Max(row => row.GetAs<double>("high"));
            //     var low = group.Min(row => row.GetAs<double>("low"));
            //     var diff = high - low;
            //     return new
            //     {
            //         Date = group.Key.Date,
            //         Interval = group.Key.Interval,
            //         PriceDiff = diff > 50 ? 1 : 0
            //     };
            // });



In [36]:
public void count_50_dollars_volatile_per_day()
{
        // Конвертуємо стовпець з датами в тип DateTime
        csv.AddColumn("DateTime", csv["Date"].Select(dateStr => DateTime.Parse(dateStr.Value.ToString())));
        // Перетворення значень нашого стовпця 'Price' в рядки та додавання їх до нового стовпця 'PriceString'
        //csv.AddColumn("PriceString", csv["Price"].Select(x => x.Value.ToString())  );


}

    count_50_dollars_volatile_per_day()

Error: System.ArgumentException: Column with a key '(Date, Exact, <StartupCode$Deedle>.$Frame+GetColumn@937)' does not exist in the data frame (Parameter 'column')
   at Deedle.Frame`2.safeGetColVector(TColumnKey column_0, Lookup column_1, FSharpFunc`2 column_2) in C:\Dev\fslab\Deedle\src\Deedle\Frame.fs:line 108
   at Deedle.Frame`2.GetColumn[R](TColumnKey column, Lookup lookup) in C:\Dev\fslab\Deedle\src\Deedle\Frame.fs:line 937
   at Deedle.Frame`2.GetColumn[R](TColumnKey column) in C:\Dev\fslab\Deedle\src\Deedle\Frame.fs:line 949
   at Deedle.Frame`2.get_Item(TColumnKey column) in C:\Dev\fslab\Deedle\src\Deedle\Frame.fs:line 903
   at Submission#37.count_50_dollars_volatile_per_day()
   at Submission#37.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchE